In [3]:
!pip install -U tensorflow_datasets

Requirement already up-to-date: tensorflow_datasets in /usr/local/lib/python3.6/dist-packages (2.1.0)


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [5]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incompleteVFOGML/fashion_mnist-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incompleteVFOGML/fashion_mnist-test.tfrecord
Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.0. Subsequent calls will reuse this data.


In [6]:
def normalize(images, label):
  print("image is {} and label is {}".format(images, label))
  images = tf.cast(images, tf.float32)
  images /= 255

  return images, label

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)


train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()


image is Tensor("args_0:0", shape=(28, 28, 1), dtype=uint8) and label is Tensor("args_1:0", shape=(), dtype=int64)
image is Tensor("args_0:0", shape=(28, 28, 1), dtype=uint8) and label is Tensor("args_1:0", shape=(), dtype=int64)


In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(128, activation=tf.nn.relu),
                             tf.keras.layers.Dense(10),
])

In [0]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [0]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [11]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4930 - accuracy: 0.8253
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3682 - accuracy: 0.8668
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3363 - accuracy: 0.8780
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3080 - accuracy: 0.8892
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2913 - accuracy: 0.8917


In [12]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 3s 9ms/step - loss: 0.3650 - accuracy: 0.8760
Accuracy on test dataset: 0.8759999871253967


In [0]:
for test_images, test_labels in test_dataset.take(20):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [28]:
predictions.shape

(32, 10)

In [30]:
predictions[0]

array([ -0.3539049 ,  -4.336967  ,   3.0949552 ,   0.36284226,
         4.529024  , -13.456577  ,   0.25837615, -12.666512  ,
        -1.6510638 ,  -8.96511   ], dtype=float32)

In [33]:
np.argmax(predictions[10])

0

In [34]:
test_labels[10]

0